In [1]:
# snp scratchpad initiation
from helper import *
from snp_func import *

# Do assignments
a = assign_var('snp')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('snp', 'live')

In [2]:
util.logToFile(logpath+'test.log')

In [3]:
# dfopt = pd.read_pickle(fspath+'targets.pickle')

In [11]:
%%time
# nse dynamic changes

#... get the open orders

open_trades = ib.openTrades()

if open_trades:
    df_open=util.df(open_trades)

    # open contracts
    df_oc = util.df(list(df_open['contract']))
    df_oc = df_oc[list(df_oc)[:6]]
    df_oc = df_oc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # orders
    df_ord = util.df(list(df_open['order']))
    df_ord = df_ord[list(df_ord)[1:8]]

    df_openord = df_oc.join(df_ord).drop('clientId', axis=1)

else:
    df_openord = pd.DataFrame() # Empty DataFrame

#.... check for overall position risk. Abort after cancelling SELL orders at risk
df_ac = util.df(ib.accountSummary())
net_liq = float(df_ac[df_ac.tag == 'NetLiquidation'][:1].value)
init_margin = float(df_ac[df_ac.tag == 'InitMarginReq'][:1].value)

if init_margin >= net_liq*ovallmarginlmt:  # if overall limit is breached
    print("Overall margin limit breached")

    # ...cancel all SELL open orders, if it is not empty
    if not df_openord.empty:
        m_opord = (df_openord.action == 'SELL')
        df_sells = df_openord[m_openord]

        # cancel the SELL openorders
        cancel_list = [o for o in ib.openOrders() if o.orderId in list(df_sells.orderId)]
        cancelled = [ib.cancelOrder(c) for c in cancel_list]

#         return None # abort the dynamic function. ***PLEASE UNCOMMENT THIS IN LIVE FUNCTION**** 

#... prepare the harvesting BUY orders for those without them

# get the positions
df_p = util.df(ib.positions())

# get the position contracts
df_pc = util.df(list(df_p.contract))
df_pc = df_pc[list(df_pc)[:6]]
df_pc = df_pc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

df_pc = df_pc.join(df_p[['position', 'avgCost']])

df_pc.head()

df_pc = df_pc.assign(avgCost=np.where(df_pc.secType == 'OPT', df_pc.avgCost/100, df_pc.avgCost))

# initialize buy orders to be equal to df_pc
df_buy = df_pc
df_buy = df_buy.assign(dte=[get_dte(d) for d in df_buy.expiration])

# get the latest prices
contracts = ib.qualifyContracts(*[Contract(conId=c) for c in df_buy.conId])
df_buy = df_buy.assign(contract=contracts)

tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {conId: optPrice}
df_buy = df_buy.assign(optPrice = df_buy.conId.map(optPrices))

# get the harvest price
df_buy = df_buy.assign(hvstPrice=[get_prec(min(hvstPricePct(d)*c,mp*0.9),0.05) for d, c, mp in zip(df_buy.dte, df_buy.avgCost, df_buy.optPrice)])

#...prepare the SELL orders

# get remaining quantities
dfrq = remqty_snp(ib)
# update target with remaining quantities
dft = pd.read_pickle(fspath+'targets.pickle')

# prepare the target
dft = dft.assign(remqty=dft.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
dft = dft[dft.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty <= 0)

# adjust optprice for fall-rise risk
risky = riskyprice(dft, 0.01)

# dft=dft.assign(expPrice = dft.optId.map(risky).fillna(dft.expPrice))

Wall time: 11.3 s


In [15]:
dft[dft.symbol=='WBA']

,symbol,expiration,strike,right,dte,margin,undPrice,lot,undId,lo52,...,loStd,hiStd,optId,optPrice,optMargin,rom,remqty,expPrice,expRom,qty
172,WBA,20190524,51.0,P,9,338,52.75,100,178634687,51.96,...,51.229942,55.030087,360027064,0.62,217.67,11.551635,9,0.67,12.483219,1
173,WBA,20190524,50.5,P,9,338,52.75,100,178634687,51.96,...,51.229942,55.030087,360027029,0.47,193.04,9.874177,9,0.52,10.924621,1
174,WBA,20190524,50.0,P,9,338,52.75,100,178634687,51.96,...,51.229942,55.030087,360027043,0.35,169.14,8.392127,9,0.40,9.591003,1


In [20]:
dft.head()

,symbol,expiration,strike,right,dte,margin,undPrice,lot,undId,lo52,...,loStd,hiStd,optId,optPrice,optMargin,rom,remqty,expPrice,expRom,qty
0,AAPL,20190621,140.0,P,37,1207,188.90,100,265598,142.00,...,150.655898,246.266152,308586955,0.29,134.80,2.122263,2,0.34,2.488171,1
1,AAPL,20190621,135.0,P,37,1207,188.90,100,265598,142.00,...,150.655898,246.266152,308586948,0.20,114.01,1.730526,2,0.25,2.163158,1
2,AAPL,20190621,130.0,P,37,1207,188.90,100,265598,142.00,...,150.655898,246.266152,308586945,0.14,94.68,1.458683,2,0.19,1.979641,1
3,ACN,20190628,130.0,P,44,1111,173.61,100,67889930,132.63,...,147.047412,213.453882,365802191,0.35,117.11,2.479215,2,0.40,2.833389,2
4,ACN,20190621,130.0,P,37,1111,173.61,100,67889930,132.63,...,147.047412,213.453882,315853195,0.20,118.86,1.659913,2,0.25,2.074892,2


In [22]:
risky[360027064]

10.89

In [19]:
dft.optId.map(risky)

0      0.19
1      0.10
2      0.06
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9      2.83
10     1.73
11     1.27
12     0.00
13      NaN
14      NaN
15      NaN
16      NaN
17     0.29
18     0.01
19      NaN
20      NaN
21      NaN
22     0.00
23     0.24
24     0.16
25     0.08
26     0.54
27     0.42
28     0.32
29     0.84
       ... 
206    0.03
207    0.01
208     NaN
209    0.12
210    0.06
211    0.04
212     NaN
213    0.07
214    0.15
215    0.03
216    0.02
217    0.98
218    0.40
219    0.18
220    0.14
221    0.08
222    0.24
223    0.12
224    0.08
225    0.13
226    0.06
227    0.09
228    0.21
229    0.14
230    0.08
231    0.02
232    0.09
233    0.04
234    0.02
235     NaN
Name: optId, Length: 236, dtype: float64

In [ ]:

if not df_openord.empty:
#... As there are some open orders, 
#    ..prepare the remaining 'BUY' orders
#    ..cancel all the 'SELL' orders to cancel
#    ..update prices and roms for dft
#    ..prepare the new harvest 'SELL' orders

    # remove the exising buys
    df_bx = df_openord[df_openord.action == 'BUY']
    df_buy = df_pc[~df_pc.conId.isin(list(df_bx.conId))]

    fill_symbols = []
    if ib.fills():     # get fill symbols, if there are any SELL fills!
        df_fc = util.df(list(util.df(ib.fills())['contract'])) # filled contracts
        df_fc = df_fc[list(df_fc)[:6]]
        df_fc = df_fc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})
        df_fc

        df_fe = util.df(list(util.df(ib.fills())['execution'])) # executed contracts
        df_fe = df_fe[list(df_fe)[4:13]].drop('liquidation', 1)

        df_fill = df_fc.join(df_fe)

        # get the fill symbols
        fill_symbols = list(df_fill.symbol.unique())

        # identify the recalc orders which needs to be cancelled and reordered
        df_rcalc = dft[dft.symbol.isin(fill_symbols)]

        # cancel the SELL orders
        sell_ordId = df_openord[df_openord.action == 'SELL'].set_index('conId')[['orderId']].to_dict()['orderId']
        df_cancel_rc = df_rcalc.assign(ordId=df_rcalc.optId.astype('int32').map(sell_ordId)).dropna()
        cancelords = [o for o in ib.openOrders() if o.orderId in list(df_cancel_rc.ordId.astype(int))]
        rc_cancelled = [ib.cancelOrder(c) for c in cancelords]

        # update and jackup expPrice to cover the risk
        df_rcalc = upd(ib, df_rcalc)
        df_rcalc = df_rcalc.assign(expPrice=get_prec(df_rcalc.expPrice*jackup,0.05))
        # re-calculate expRom
        df_rcalc = df_rcalc.assign(expRom=df_rcalc.expPrice*df_rcalc.lot/df_rcalc.margin*365/df_rcalc.dte)
        
        # remove the recalc optIds from dft
        dft=dft[~dft.optId.isin(list(df_rcalc.optId))]

        # re-add df_rcalc to dft and pickle
        dft = grp_opts(pd.concat([dft, df_rcalc[dft.columns]]))

        # write to pickle
        dft.to_pickle((fspath+'targets.pickle'))

#...prepare to place the orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                      for position, hvstPrice in zip(df_buy.position, df_buy.hvstPrice)]
hco = zip(contracts, buy_orders)
sell_blks = trade_blocks(ib, dft, exchange=exchange)

In [ ]:
def riskyprice(df, prec):
    '''adjusts expPrice to accomodate risk
    Args:
        (df) as the options dataframe
        (prec) precision needed as int'''
    pmask = (dft.right == 'P') & (dft.undPrice-dft.strike-dft.expPrice < dft.Fall)
    df_prisky = pd.merge(dft[pmask][['symbol', 'optId', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom', 'qty']], 
             pd.DataFrame((dft[pmask].undPrice-dft[pmask].strike-dft[pmask].expPrice)), on=dft[pmask].index).drop('key_0', 1)

    cmask = (dft.right == 'C') & (dft.strike-dft.undPrice-dft.expPrice < dft.Rise)
    df_crisky = pd.merge(dft[cmask][['symbol', 'optId', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom', 'qty']], 
             pd.DataFrame((dft[cmask].strike-dft[cmask].undPrice-dft[cmask].expPrice)), on=dft[cmask].index).drop('key_0', 1)

    df_risky = pd.concat([df_prisky, df_crisky]).reset_index(drop=True)

    df_risky = df_risky.rename(columns={0: 'FallRise'})

    df_risky = df_risky.assign(FRpct=abs(np.where(df_risky.right == 'P', (df_risky.FallRise - df_risky.Fall)/df_risky.FallRise, (df_risky.FallRise - df_risky.Rise)/df_risky.FallRise)))

    df_risky = df_risky.sort_values('FRpct', ascending=False)

    df_risky = df_risky.assign(expPrice = get_prec(df_risky.FRpct*df_risky.expPrice, prec))

    return df_risky.set_index('optId')['optPrice'].to_dict()